In [ ]:
!pip install nltk
!pip install wordnet
# !pip install negspacy
!unzip /usr/share/nltk_data/corpora/wordnet.zip -d /usr/share/nltk_data/corpora/
!pip install gensim
!pip install torch-optimizer

In [ ]:
!wget https://nlp.stanford.edu/data/glove.twitter.27B.zip
!unzip -q glove.twitter*.zip

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
import torch
import torch.nn as nn #Base class for all neural network modules.
import matplotlib.pyplot as plt
import optuna
# import spacy
import gensim
import re
import random
import torch_optimizer as optim
import copy

from nltk.tokenize import TweetTokenizer
from nltk.tokenize import word_tokenize
from gensim.models import KeyedVectors
# from negspacy.negation import Negex
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.pipeline import Pipeline
from torch.utils.data import TensorDataset, DataLoader
from torch.optim.lr_scheduler import ReduceLROnPlateau
from collections import Counter

# Set a random seed for reproducibility
SEED = 42  

# Python's built-in random
random.seed(SEED)

# Numpy
np.random.seed(SEED)

# PyTorch
torch.manual_seed(SEED)
torch.backends.cudnn.deterministic = True 
torch.backends.cudnn.benchmark = False

# Για PyTorch με CUDA (GPU)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(SEED)


In [ ]:
train_data = pd.read_csv('/kaggle/input/ai-2-dl-for-nlp-2025-homework-2/train_dataset.csv')
test_data = pd.read_csv('/kaggle/input/ai-2-dl-for-nlp-2025-homework-2/test_dataset.csv')
val_data = pd.read_csv('/kaggle/input/ai-2-dl-for-nlp-2025-homework-2/val_dataset.csv')

In [ ]:

# spacy.prefer_gpu()
# nlp = spacy.load("en_core_web_sm")

# NEGATION_WORDS = {"no", "not", "none", "neither", "nor", "never"}
# NEGATION_PHRASES = {
#     "not at all": "not_at_all",
#     "not only": "not_only"
# }
# def handle_negation(text):
#     """Επεξεργάζεται την άρνηση στο κείμενο χωρίς να προσθέτει το 'not_' μπροστά από τις λέξεις."""
#     words = word_tokenize(text.lower())  # Χρησιμοποιούμε το word_tokenize για να πάρουμε τις λέξεις
#     processed_words = []
#     negated = False  # Εντοπιστής άρνησης

#     for i, word in enumerate(words):
#         # Ελέγχουμε αν η λέξη είναι 'n't' ή 'no', τα οποία υποστηρίζονται από το GloVe
#         if word in NEGATION_WORDS:
#             negated = True
#             processed_words.append(word)  # Δεν προσθέτουμε το 'not_' γιατί το GloVe το αναγνωρίζει
#         elif word in NEGATION_PHRASES:  # Ελέγχουμε αν υπάρχει φράση άρνησης
#             processed_words.append(f"not_{word.replace(' ', '_')}")  # Κρατάμε την φράση με 'not_'
#             negated = False  # Άρνηση τελείωσε
#         elif negated:
#             # Αν έχουμε άρνηση και δεν βρούμε 'n't' ή 'no', προσθέτουμε το 'not_' πριν τη λέξη
#             processed_words.append(f"not_{word}")
#             negated = False  # Τελείωσε η άρνηση
#         else:
#             processed_words.append(word)

#     return " ".join(processed_words)

def replace_emoticons(text):
    """
    Replaces common emoticons with text tokens preserving sentiment.
    Improves accuracy by maintaining emotional context.
    """
    # Emoticon-to-text mapping dictionary
    EMOTICON_MAP = {
        # Positive
        ":‑)": " happy ", ":)": " happy ", ":-]": " happy ", 
        ":]": " happy ", ":-3": " happy ", ":3": " happy ",
        "=]": " happy ", "=)": " happy ", ":)": " happy ",
        "=D": " happy", '<3': ' love ', 
        
        # Negative
        "=(": " sad ", ":(": " sad ", ":-c": " sad ",
        ":-[": " sad ", ":[": " sad ", ":{": " sad ",
    }
    
    # Replace emoticons
    for emoticon, replacement in EMOTICON_MAP.items():
        text = text.replace(emoticon, replacement)
    
    # Handle repeated characters ("Heyyyy" -> "Heyy")
    text = re.sub(r'(.)\1{2,}', r'\1\1', text)
    
    # Clean whitespace
    text = re.sub(r'\s+', ' ', text).strip()
    
    return text

def replace_social_sentiment(text):
    """
    Replaces social media interjections with corresponding sentiment tokens.
    Returns original text if no matches found.
    """
    if not isinstance(text, str) or not text.strip():
        return text  # Return original for non-text
    
    SOCIAL_PATTERNS = {
        r"\bhahaha+\b": "happy",
        r"\bhehe+\b": "happy",
        r"\blol\b": "happy",
        r"\blmao\b": "happy",
        r"\bbruh+\b": "awkward",
        r"\boof\b": "awkward",
        r"\bwow\b": "surprise",
        r"\bomg\b": "surprise",
        r"\bugh\b": "annoyed",
        r"\bsmh\b": "disappointed"
    }

    modified = False
    new_text = text.lower()  # Work in lowercase for consistency

    for pattern, replacement in SOCIAL_PATTERNS.items():
        if re.search(pattern, new_text):
            new_text = re.sub(pattern, replacement, new_text)
            modified = True

    return new_text if modified else text

def remove_unicode(text):
    """Convert to ASCII and clean special characters"""
    text = text.encode('ascii', 'ignore').decode('ascii')
    # Additional cleaning
    text = re.sub(r"[^\w\s.,!?;]", "", text)
    text = re.sub(r"\s+", " ", text).strip()
    return text

def preprocess_text(text):
 
    # Unicode normalization
    text = remove_unicode(text)
    # Replace emoticons
    text = replace_emoticons(text)
    # Replace social expressions
    text = replace_social_sentiment(text)
    # Lowercase
    text = text.lower()
    # URLs → <url>
    text = re.sub(r"http\S+|www\.\S+", "<url>", text)
    # Mentions → <user>
    text = re.sub(r"@\w+", "<user>", text)
    # Numbers → <number>
    text = re.sub(r"\d+", "<number>", text)
    # text = re.sub(r'\d+', '', text)
    #Normalize repeated punctuation (!! → !)
    text = re.sub(r"([!?.,])\1{1,}", r"\1", text)
    
    return text

    # text = text.translate(str.maketrans('', '', string.punctuation)) 
    
    # # Correct the spelling mistakes
    # text = re.sub(r"\b(luv)\b", "love", text)            
    # text = re.sub(r"\b(amzing)\b", "amazing", text)
    # text = re.sub(r"\b(terible)\b", "terrible", text)
    # text = re.sub(r"\b(excelent)\b", "excellent", text)
    # text = re.sub(r"\b(perfonmence)\b", "performance", text)
    # text = re.sub(r"\b(gud)\b", "good", text)
    # text = re.sub(r"\b(vry)\b", "very", text)
    # text = re.sub(r"\b(fanstic)\b", "fantastic", text)
    # text = re.sub(r"\b(gr8)\b", "great", text)
    # text = re.sub(r"\b(horrble)\b", "horrible", text)
    # text = re.sub(r"\b(u)\b", "you", text)
    # text = re.sub(r"\b(guyz)\b", "guys", text)
    # text = re.sub(r"\b(knw)\b", "know", text)
    # text = re.sub(r"\b(da)\b", "the", text)
    # text = re.sub(r"\b(btw)\b", "by the way", text)
    # text = re.sub(r"\b(r)\b", "are", text)
    # text = re.sub(r"\b(cuz)\b", "because", text)
    # text = re.sub(r"\b(tho)\b", "though", text)
    # text = re.sub(r"\b(lol)\b", "laugh out loud", text)
    # text = re.sub(r"\b(ur)\b", "your", text)


    
    # tokens = tweet_tokenizer.tokenize(text)  # Tokenization using TweetTokenizer

    # return " ".join(tokens)  # Return processed text as string

    # #Tokenization
    # words = word_tokenize(text)
    # return " ".join(words)

    # Remove custom stopwords or 
    # Remove stopwords but keep negations
    # words = [word for word in words if word not in stop_words]
    # processed_text = ' '.join(words)
    
    # words = [word for word in words if word not in custom_stopwords]

    # Lemmatization 
    # lemmatized_words = [lemmatizer.lemmatize(word) for word in words]
    # processed_text = ' '.join(lemmatized_words)
    # return processed_text

    # tagged_words = pos_tag(words)
    # # Keep only nouns (NN) and verbs (VB) //no
    # filtered_words = [word for word, tag in tagged_words if tag.startswith('NN') or tag.startswith('VB')]
    # # Rejoin words into text
    # processed_text = ' '.join(filtered_words)
    # return processed_text

    # # Stemming
    # stemmed_words = [stemmer.stem(word) for word in words] #probably not
    # # Join words back into a single string
    # processed_text = ' '.join(stemmed_words)
    # return processed_text


# def correct_spelling(text):
#     blob = TextBlob(text)
#     corrected_text = str(blob.correct())
#     return corrected_text


# Spelling correction
# train_data['Text'] = train_data['Text'].apply(correct_spelling)
# test_data['Text'] = test_data['Text'].apply(correct_spelling)
# val_data['Text'] = val_data['Text'].apply(correct_spelling)

In [ ]:
train_data['clean_text'] = train_data['Text'].apply(preprocess_text)
val_data['clean_text'] = val_data['Text'].apply(preprocess_text)

In [ ]:
# Create a function to find most frequent out-of-vocabulary (OOV) words
def get_oov_words(texts, glove_vocab, top_k=50):
    """Identify top K most frequent words not in GloVe vocabulary"""
    all_tokens = []
    # Tokenize all texts and collect tokens
    for text in texts:
        tokens = preprocess_text(text).split()
        all_tokens.extend(tokens)
    
    # Count word frequencies
    counter = Counter(all_tokens)
    # Filter words not in GloVe vocabulary
    oov_words = {word: freq for word, freq in counter.items() if word not in glove_vocab}
    # Sort by frequency (descending)
    sorted_oov = sorted(oov_words.items(), key=lambda x: -x[1])
    return sorted_oov[:top_k]  # Return top K most frequent OOV words

# Load GloVe vocabulary (one-time operation)
glove_path = "glove.twitter.27B.200d.txt"  # Update path as needed

glove_vocab = set()  # Store unique words from GloVe
with open(glove_path, "r", encoding="utf8") as f:
    for line in f:
        word = line.split(" ")[0]  # First element is the word
        glove_vocab.add(word)  # Add to vocabulary set

# Find top OOV words in training and validation sets
oov_train = get_oov_words(train_data['clean_text'], glove_vocab)
oov_val = get_oov_words(val_data['clean_text'], glove_vocab)

# Print results
print("Top OOV words in train:")
for word, freq in oov_train:
    print(f"{word}: {freq}")  # word: frequency count

print("\nTop OOV words in val:")
for word, freq in oov_val:
    print(f"{word}: {freq}")

In [ ]:
# word2vec_model = api.load("word2vec-google-news-300")
# glove_input_file = 'glove.6B.200d.txt'
glove_input_file = "glove.twitter.27B.200d.txt"

model = KeyedVectors.load_word2vec_format(glove_input_file, binary=False, no_header=True)


# # Δείγμα 20 πρώτων λέξεων
# w2v_dict = {
#     'word': [],
#     'vector': []
# }

# for i, word in enumerate(model.index_to_key[:50]):
#     w2v_dict['word'].append(word)
#     w2v_dict['vector'].append(model[word])


# w2v_df = pd.DataFrame.from_dict(w2v_dict)
# w2v_df

In [ ]:
# Initialize tweet tokenizer for splitting text into tokens
tweet_tokenizer = TweetTokenizer()

def text_to_embedding(text, embedding_model=model, embedding_dim=200):
    """Convert text to embedding by averaging token vectors"""
    
    # Tokenize input text
    tokens = tweet_tokenizer.tokenize(text)
    
    # Get vectors for each token
    vectors = []
    for token in tokens:
        if token in embedding_model:  # Check if token exists in embedding model
            vectors.append(embedding_model[token])  # Use pre-trained embedding
        else:
            vectors.append(np.zeros(embedding_dim))  # Zero vector for out-of-vocabulary tokens
    
    # Return average of all token vectors
    if vectors:
        return np.mean(vectors, axis=0)  # Average pooling
    else:
        return np.zeros(embedding_dim)  # Fallback for empty text

# Apply embedding function to datasets
train_data['vector'] = train_data['clean_text'].apply(text_to_embedding)  # Embed training texts
val_data['vector'] = val_data['clean_text'].apply(text_to_embedding)  # Embed validation texts

In [ ]:

# Convert embeddings to PyTorch tensors
# For training set
x_train = torch.tensor(np.stack(train_data['vector'].values), dtype=torch.float32)  # Stack embeddings into matrix
y_train = torch.tensor(train_data['Label'].values, dtype=torch.float32)  # Convert labels to tensor

# For validation set
x_val = torch.tensor(np.stack(val_data['vector'].values), dtype=torch.float32)
y_val = torch.tensor(val_data['Label'].values, dtype=torch.float32)

# Create PyTorch Dataset objects
train_dataset = TensorDataset(x_train, y_train)  # (features, labels) pairs
val_dataset = TensorDataset(x_val, y_val)

# Print tensor shapes for verification
print(f"x_train shape: {x_train.shape}")  # [num_samples, embedding_dimension]
print(f"y_train shape: {y_train.shape}")  # [num_samples]
print(f"x_val shape: {x_val.shape}")
print(f"y_val shape: {y_val.shape}")

In [ ]:
class DynamicNet(nn.Module):
    def __init__(self, D_in, H1, H2, D_out=1, num_layers=2, dropout=0.3, batch_norm=False):
        super(DynamicNet, self).__init__()
        self.layers = nn.ModuleList()  # Flexible layer container

        # First layer (input to first hidden)
        self.layers.append(nn.Linear(D_in, H1))
        if batch_norm:
            self.layers.append(nn.BatchNorm1d(H1))  # Normalization
        self.layers.append(nn.GELU())  # Activation
        self.layers.append(nn.Dropout(dropout))  # Regularization

        # Additional hidden layers
        current_dim = H1
        for _ in range(num_layers - 1):
            self.layers.append(nn.Linear(current_dim, H2))
            if batch_norm:
                self.layers.append(nn.BatchNorm1d(H2))
            self.layers.append(nn.GELU())
            self.layers.append(nn.Dropout(dropout))
            current_dim = H2  # Update dimension for next layer

        # Final output layer (no activation - handled by loss)
        self.layers.append(nn.Linear(current_dim, D_out))

    def forward(self, x):
        for layer in self.layers:
            x = layer(x)  # Sequential processing
        return x  # Raw logits for BCEWithLogitsLoss

# Model configuration (tuned via Optuna)
D_in = x_train.shape[1]  # Input dimension from data
H1 = 344  # First hidden layer size (optimized)
H2 = 224  # Second hidden layer size (optimized)
D_out = 1  # Binary classification output
num_layers = 2  # Total hidden layers
dropout = 0.3  # Dropout rate (found optimal)
use_batch_norm = True  # Batch normalization flag

# Training parameters (optimized)
learning_rate = 0.00012538928471634227  # LR from Optuna
weight_d = 0.000  # L2 regularization (disabled)
batch_size = 128  # Batch size (optimized)

# Initialize model with best parameters
model = DynamicNet(D_in, H1, H2, D_out, num_layers, dropout, use_batch_norm)

# Loss function (combines sigmoid + BCE)
loss_func = nn.BCEWithLogitsLoss()

# Adam optimizer with tuned learning rate
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate, weight_decay=weight_d)

# LR scheduler (reduces LR on plateau)
scheduler = ReduceLROnPlateau(optimizer, mode='min', patience=3, factor=0.5)

# Prepare datasets and loaders
train_dataset = TensorDataset(x_train, y_train)
val_dataset = TensorDataset(x_val, y_val)

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size)

model

In [ ]:
# Initialize tracking variables
train_losses = []
val_losses = []
train_accuracies = []
val_accuracies = []

best_val_loss = float('inf')  # Track best validation loss
patience_counter = 0
best_model_state = None
patience = 5  # Early stopping patience
epochs = 30  # Max epochs

# Training phase
for epoch in range(epochs):
    model.train()
    train_loss = 0.0
    all_train_preds = []
    all_train_labels = []

    # Forward pass + backprop
    for x_batch, y_batch in train_loader:
        y_batch = y_batch.view(-1, 1).float()
        optimizer.zero_grad()
        logits = model(x_batch)
        loss = loss_func(logits, y_batch)
        loss.backward()
        optimizer.step()
        train_loss += loss.item()
        all_train_preds.append((torch.sigmoid(logits) >= 0.5).int())
        all_train_labels.append(y_batch.int())

    # Calculate epoch training metrics
    avg_train_loss = train_loss / len(train_loader)
    train_preds = torch.cat(all_train_preds).squeeze()
    train_labels = torch.cat(all_train_labels).squeeze()
    train_accuracy = accuracy_score(train_labels.cpu(), train_preds.cpu())
    
 # VALIDATION
    model.eval()
    val_loss = 0.0
    all_val_preds = []
    all_val_labels = []

    with torch.no_grad():
        for x_val_batch, y_val_batch in val_loader:
            y_val_batch = y_val_batch.view(-1, 1).float()
            val_logits = model(x_val_batch)
            loss = loss_func(val_logits, y_val_batch)
            val_loss += loss.item()
            all_val_preds.append((torch.sigmoid(val_logits) >= 0.5).int())
            all_val_labels.append(y_val_batch.int())

    # Calculate epoch validation metrics 
    avg_val_loss = val_loss / len(val_loader)
    val_preds = torch.cat(all_val_preds).squeeze()
    val_labels = torch.cat(all_val_labels).squeeze()
    val_accuracy = accuracy_score(val_labels.cpu(), val_preds.cpu())

    train_losses.append(avg_train_loss)
    val_losses.append(avg_val_loss)
    train_accuracies.append(train_accuracy)
    val_accuracies.append(val_accuracy)

    # Store metrics
    print(f"Epoch {epoch+1}/{epochs} | Train Loss: {avg_train_loss:.4f} | Val Loss: {avg_val_loss:.4f} | "
          f"Train Acc: {train_accuracy:.4f} | Val Acc: {val_accuracy:.4f}")

    # Update learning rate scheduler
    scheduler.step(avg_val_loss)

     # Early stopping check
    if avg_val_loss < best_val_loss:
        best_val_loss = avg_val_loss
        best_model_state = copy.deepcopy(model.state_dict())
        patience_counter = 0
    else:
        patience_counter += 1
        print(f"No improvement. Patience: {patience_counter}/{patience}")
        if patience_counter >= patience:
            print("Early stopping triggered!")
            break

# Restore best model
if best_model_state:
    model.load_state_dict(best_model_state)

In [ ]:
# Evaluation
model.eval()  # Set model to evaluation mode
with torch.no_grad():
    logits = model(x_val)
    probs = torch.sigmoid(logits)  # Convert logits to probabilities
    y_val_pred = (probs >= 0.5).int().squeeze()  # Convert to binary predictions (0/1)

# Print validation metrics
print("Validation Metrics:")
print("------------------")
print(f"Accuracy:  {accuracy_score(y_val, y_val_pred):.6f}")  # Classification accuracy
print(f"Precision: {precision_score(y_val, y_val_pred):.6f}")  # Positive predictive value
print(f"Recall:    {recall_score(y_val, y_val_pred):.6f}")     # True positive rate
print(f"F1 Score:  {f1_score(y_val, y_val_pred):.6f}")         # Harmonic mean of precision/recall
print()

# Training set evaluation
with torch.no_grad():
    logits = model(x_train)
    probs = torch.sigmoid(logits)
    y_train_pred = (probs >= 0.5).int().squeeze()

print("Training Metrics:")
print("------------------")
print(f"Accuracy:  {accuracy_score(y_train, y_train_pred):.6f}")  # Model performance on training data

In [ ]:
plt.figure(figsize=(8, 6))

# Define data and x-axis range based on training duration
epochs_range = range(1, len(train_losses) + 1)  # Automatically covers all epochs (e.g., 25)

# Plot loss curves (blue and green)
plt.plot(epochs_range, train_losses, label='Training Loss', color='blue', linestyle='-', linewidth=2)
plt.plot(epochs_range, val_losses, label='Validation Loss', color='green', linestyle='-', linewidth=2)

# Plot accuracy curves (red and orange)
plt.plot(epochs_range, train_accuracies, label='Training Accuracy', color='red', linestyle='-', linewidth=2)
plt.plot(epochs_range, val_accuracies, label='Validation Accuracy', color='orange', linestyle='-', linewidth=2)

# Titles and axes
plt.title('Learning Curves', fontsize=14, pad=20)
plt.xlabel('Epoch', fontsize=12)
plt.ylabel('Metric Value', fontsize=12)
plt.xticks(epochs_range[::3])  # Show every 3rd epoch 
plt.yticks(np.arange(0.2, 1.1, 0.1))  # Y-axis ticks from 0.2 to 1.0 in 0.1 steps
plt.ylim(0.2, 1.0)  # Fixed y-range for accuracy (adjust if loss exceeds 1.0)

# Legend and grid
plt.legend(loc='upper right', framealpha=1.0)
plt.grid(True, linestyle='--', alpha=0.7)

# Display
plt.tight_layout()
plt.show()

In [ ]:
from sklearn.metrics import roc_curve, auc, RocCurveDisplay

# Calculate and plot ROC curve
val_probs = []  # Model's predicted probabilities
val_true_labels = []  # Actual labels

with torch.no_grad():
    for x_val_batch, y_val_batch in val_loader:
        logits = model(x_val_batch)
        val_probs.extend(torch.sigmoid(logits).cpu().numpy())
        val_true_labels.extend(y_val_batch.float().cpu().numpy())

# Compute ROC metrics
fpr, tpr, _ = roc_curve(val_true_labels, val_probs)
roc_auc = auc(fpr, tpr)

# Plot configuration
plt.figure(figsize=(8,6))
plt.plot(fpr, tpr, label=f'AUC={roc_auc:.2f}')  # Model's performance
plt.plot([0,1],[0,1], 'k--')  # Random baseline
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.legend()
plt.show()

In [ ]:
# def suggest_hyperparameters(trial):
#     H1 = trial.suggest_int("H1", 120, 376, step=32)
#     H2 = trial.suggest_int("H2", 64, 256, step=32)
#     num_layers = trial.suggest_int("num_layers", 2, 3)

#     # Create parameter dictionary with all tunable hyperparameters
#     params = {
#         "H1": H1, "H2": H2, "num_layers": num_layers,
#         "optimizer": trial.suggest_categorical("optimizer", ["Adam", "AdamW"]),
#         "lr": trial.suggest_float("lr", 1e-4, 1e-3, log=True),
#         "batch_size": trial.suggest_categorical("batch_size", [64, 128]),
#         "weight_d": trial.suggest_categorical("weight_d", [0.0, 1e-6, 1e-5, 1e-4]),
#         "dropout": trial.suggest_float("dropout", 0.1, 0.5, step=0.1),
#         "batch_norm": trial.suggest_categorical("batch_norm", [True])
#     }

#     return params

In [ ]:
# # Set device to GPU if available, otherwise CPU
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# def objective(trial):
#     # Get hyperparameters from Optuna trial
#     params = suggest_hyperparameters(trial)
    
#     # Create data loaders with current batch size
#     train_loader = DataLoader(train_dataset, batch_size=params["batch_size"], shuffle=True)
#     val_loader = DataLoader(val_dataset, batch_size=params["batch_size"])
    
#     # Initialize model with dynamic architecture based on trial parameters
#     D_in = x_train.shape[1]  # Input dimension from training data
#     model = DynamicNet(
#         D_in=D_in, D_out=1,  # Binary classification output
#         H1=params["H1"], H2=params["H2"],  # Hidden layer sizes
#         num_layers=params["num_layers"],  # Number of hidden layers
#         dropout=params["dropout"],  # Dropout rate
#         batch_norm=params["batch_norm"]  # Batch normalization flag
#     ).to(device)  # Move model to GPU if available
    
#     # Configure optimizer based on trial suggestion
#     optimizer_classes = {
#         "Adam": torch.optim.Adam,
#         "AdamW": torch.optim.AdamW
#     }
#     optimizer = optimizer_classes[params["optimizer"]](
#         model.parameters(), 
#         lr=params["lr"],  # Learning rate
#         weight_decay=params["weight_d"]  # L2 regularization
#     )
    
#     # Loss function and learning rate scheduler
#     loss_func = nn.BCEWithLogitsLoss()  # Binary cross-entropy with logits
#     scheduler = ReduceLROnPlateau(optimizer, mode='min', patience=3, factor=0.5)
    
#     # Early stopping configuration
#     patience = 5
#     best_val_loss = float('inf')
#     patience_counter = 0
#     best_model_state = None
#     max_epochs = 30
    
#     # Training loop
#     for epoch in range(max_epochs):
#         model.train()
#         train_loss = 0.0
        
#         # Batch training
#         for x_batch, y_batch in train_loader:
#             x_batch = x_batch.to(device)
#             y_batch = y_batch.view(-1, 1).float().to(device)
            
#             optimizer.zero_grad()
#             logits = model(x_batch)
#             loss = loss_func(logits, y_batch)
#             loss.backward()
#             optimizer.step()
#             train_loss += loss.item()
        
#         # Validation phase
#         model.eval()
#         val_loss = 0.0
#         with torch.no_grad():
#             for x_val_batch, y_val_batch in val_loader:
#                 x_val_batch = x_val_batch.to(device)
#                 y_val_batch = y_val_batch.view(-1, 1).float().to(device)
                
#                 val_logits = model(x_val_batch)
#                 loss = loss_func(val_logits, y_val_batch)
#                 val_loss += loss.item()
        
#         avg_val_loss = val_loss / len(val_loader)
#         scheduler.step(avg_val_loss)  # Adjust learning rate
        
#         # Early stopping check
#         if avg_val_loss < best_val_loss:
#             best_val_loss = avg_val_loss
#             patience_counter = 0
#             best_model_state = copy.deepcopy(model.state_dict())
#         else:
#             patience_counter += 1
#             if patience_counter >= patience:
#                 break
    
#     # Load best model state before evaluation
#     if best_model_state:
#         model.load_state_dict(best_model_state)
    
#     # Final evaluation on validation set
#     model.eval()
#     with torch.no_grad():
#         x_val_gpu = x_val.to(device)
#         logits = model(x_val_gpu)
#         probs = torch.sigmoid(logits)
#         y_pred = (probs >= 0.5).int().cpu().squeeze()
    
#     acc = accuracy_score(y_val.int(), y_pred)
#     return acc  # Return accuracy for Optuna optimization

# # Configure Optuna study
# study = optuna.create_study(
#     direction="maximize",  # We want to maximize accuracy
#     study_name="advanced_nn_tuning",
#     sampler=optuna.samplers.TPESampler(),  # Tree-structured Parzen Estimator
#     pruner=optuna.pruners.HyperbandPruner()  # Early pruning of unpromising trials
# )

# # Run optimization
# study.optimize(objective, n_trials=20, timeout=3600)  # 20 trials or 1 hour max

# # Print results
# print("Best Params:", study.best_trial.params)
# print("Best Accuracy:", study.best_value)

In [ ]:
test_data['clean_text'] = test_data['Text'].apply(preprocess_text)
test_data['vector'] = test_data['clean_text'].apply(text_to_embedding)

x_test =  torch.tensor(np.stack(test_data['vector'].values), dtype=torch.float32)
model.eval()  
with torch.no_grad():
    logits = model(x_test)
    probs = torch.sigmoid(logits)
    test_predictions = (probs >= 0.5).int().squeeze()

submission = pd.DataFrame({
    'ID': test_data['ID'],  
    'Label': test_predictions
})

submission.to_csv('submission.csv', index=False)
print("Submission file created successfully!")
print(submission.head())
